# Transmission assumptions
In the previous chapters, 
we have been using frequency-dependent transmission assumption.
Under this assumption, the rate of transmission is proportional
to the number of susceptibles and the prevalence of infectious persons in the population.
We have been referring to the parameter used for this transition process as the `contact_rate`,
in which case we can consider this as the _per capita_ rate at which
two specific individuals come into effective contact.
Note that an effective contact can be defined as a contact that would result in
transmission were it to occur between a susceptible person and an infectious person.
Let's return to a really simple SIR model.
In fact, even simpler than the one we introduced in notebook 02,
because it doesn't even have an infection-related death flow.

In [ ]:
# If running on Google Colab, run the following line of code to install the summer package
# %pip install summerepi2

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def get_sir_base_structure(
    model_config: dict,
) -> CompartmentalModel:
    """
    Generate a mode that doesn't do much in itself, but has basic
    characteristics that we can then use to add our transmission assumptions to.
    
    Args:
        model_config: The fixed values used in creating the model structure
    Returns:
        The summer model object
    """
    
    # Compartments are comprised of the base ones and any additional latency compartments requested
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    
    # Otherwise the model is very similar to that from notebook 01, as follows
    infectious_compartment = ["infectious"]
    analysis_times = (
        model_config["start_time"], 
        model_config["end_time"],
    )
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=infectious_compartment,
    )
    pop = model_config["population"]
    seed = model_config["seed"]
    suscept_pop = pop - seed
    msg = "Seed larger than population"
    assert pop >= 0., msg
    model.set_initial_population(
        distribution={
            "susceptible": suscept_pop, 
            "infectious": seed}
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("recovery"), 
        source="infectious", 
        dest="recovered",
    )
    return model

In [ ]:
config = {
    "population": 1.,
    "seed": 0.01,
    "start_time": 0.,
    "end_time": 20.,
}
freq_parameters = {
    "recovery": 0.333,
    "contact_rate": 1.,
}

In [ ]:
sir_freq_model = get_sir_base_structure(config)
sir_freq_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"),
    source="susceptible", 
    dest="infectious",
)
sir_freq_model.run(parameters=freq_parameters)
compartment_values = sir_freq_model.get_outputs_df()
compartment_values.plot()

Let's compare density-dependent transmission to frequency-dependent transmission.
The fundamental difference here is that the force of infection
is proportional to the number of infectious persons rather than the
prevalence of infectious persons.
For all our previous models,
and as briefly shown in the ordinary differential equations presented in notebook 07,
we have previously assumed that the force of infection scales according
to the prevalence of infectious individuals in the simulated population.

That is, frequency-dependent transmission:

$$ \lambda \propto \frac{I(t)}{N(t)} $$

where $\lambda$ represents the force of infection,
$I(t)$ the number of infectious persons in the population,
$N(t)$ the total population size
and $t$ time.

Whereas for density-dependent transmission:

$$ \lambda \propto I(t) $$

For either of these assumptions, 
the force of infection is then multiplied by the source compartment
(`susceptible`) when we come to actually calculating the number of people being infected.
In this way, we can think of the force of infection in a similar way to 
how we think of a parameter to a standard inter-compartmental transition flow
(the difference being that the rate is dependent on quantity that emerges from the model).

Next, let's run a model with density-dependent transmission instead of frequency-dependent.

In [ ]:
dens_parameters = {
    "recovery": 0.333,
    "contact_rate": 1.,
}
sir_dens_model = get_sir_base_structure(config)
sir_dens_model.add_infection_density_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"),
    source="susceptible", 
    dest="infectious",
)
sir_dens_model.run(parameters=dens_parameters)
compartment_values = sir_dens_model.get_outputs_df()
compartment_values.plot()

The model outputs are identical.
This is unsurprising because this is a really trivial example.
In this model the population size is fixed at one,
so the division by $N(t)$ makes no different to the force of infection.
However, next let's consider what happens if we scale the population size back up to 1000.
We can easily recover the exact same dynamics as we saw previously for both the
frequency and density-dependent models by dividing `population`, `seed` and `contact_rate`
by the new, larger population size.

In [ ]:
config.update(
    {
        "population": 1000.,
        "seed": 10.,
    }
)
dens_parameters.update(
    {
        "contact_rate": 0.001,
    }
)
large_pop_dens_model = get_sir_base_structure(config)
large_pop_dens_model.add_infection_density_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"),
    source="susceptible", 
    dest="infectious",
)
large_pop_dens_model.run(parameters=dens_parameters)
compartment_values = large_pop_dens_model.get_outputs_df()
compartment_values.plot()

So here we have shown that if the population size is fixed over time,
we can easily recover the same dynamics for both transmission types.
The only difference is the interpretation of the `contact_rate` parameter
that we used to produce these two simulations.
Under the assumption of frequency dependence,
we can interpret this parameter as the rate at which two specific individuals
come into effective contact in the population
(recalling that effective contact is a contact which would result in transmission
if it were to occur between a susceptible and an infectious person).
By contrast, under the assumption of density dependence,
the `contact_rate` parameter should be interpreted as the number of effective contacts
made by an infectious person per unit time.
Therefore, the parameter needs to be smaller by a factor of the size of 
the population under density dependence compared to frequency dependence.

So far, this is all pretty trivial.
We have demonstrated identical dynamics under both sets of assumptions.
Therefore, let's consider what would happen if the population size changes over time.
To do this, let's add deaths to all of the compartments so that the population
changes dramatically during the simulation time frame.
This unrealistic if we're thinking of the time unit as days,
but is an easy way to make sure the population size changes rapidly enough.

In [ ]:
sir_freq_births_model = get_sir_base_structure(config)
sir_freq_births_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"),
    source="susceptible", 
    dest="infectious",
)
freq_parameters.update(
    {
        "crude_death_rate": 0.05,
    }
)
sir_freq_births_model.add_universal_death_flows(
    "non_infection_deaths",
    Parameter("crude_death_rate"),
)
sir_freq_births_model.run(parameters=freq_parameters)
compartment_values = sir_freq_births_model.get_outputs_df()
compartment_values.plot(
    title="Frequency dependence, declining population"
)

In [ ]:
# Get the proportions of the population by compartment, rather than absolute numbers
compartment_values.div(compartment_values.sum(axis=1), axis=0).plot(
    title="Frequency dependence, unchanged proportional epidemic dynamics"
)

In [ ]:
sir_dens_births_model = get_sir_base_structure(config)
sir_dens_births_model.add_infection_density_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"),
    source="susceptible", 
    dest="infectious",
)
dens_parameters.update(
    {
        "crude_death_rate": 0.05,
    }
)
sir_dens_births_model.add_universal_death_flows(
    "non_infection_deaths",
    Parameter("crude_death_rate"),
)
sir_dens_births_model.run(parameters=dens_parameters)
compartment_values = sir_dens_births_model.get_outputs_df()
compartment_values.plot(
    title="Density dependence, declining population"
)

In [ ]:
compartment_values.div(compartment_values.sum(axis=1), axis=0).plot(
    title="Density dependence, different infection dynamics as population changes"
)